Objective: Perform dimenstion reduction on Titanic dataset features using PCA, with the goal of using the first several principal components in a KNN classifer.

# Imports

In [1]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

# Obtain

Load train & test data as dataframes from csv files

In [2]:
# Import train and test data using pandas

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Scrub

In [5]:
# Combine train and test data into one dataframe

train = train.drop('Survived', axis=1)
df = pd.concat([train, test], axis=0, ignore_index=True)

In [6]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1304,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
1308,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


Check each feature for missing data and data types

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


One hot encode Sex feature

In [9]:
s_dummy = pd.get_dummies(df.Sex, drop_first=True, prefix='sex')
df = pd.concat([df, s_dummy], axis=1)
df = df.drop('Sex', axis=1)

Fill Embarked with most common value, S, and one-hot encode

In [11]:
df.Embarked.fillna('S', inplace=True)
e_dummy = pd.get_dummies(df.Embarked, drop_first=False, prefix='emb')
df = pd.concat([df, e_dummy], axis=1)
df = df.drop('Embarked', axis=1)

Fill missing fare value with 0

In [13]:
df.Fare.fillna(0, inplace=True)

Convert Cabin to a binary feature, with value '1' if cabin is listed, and '0' if it isn't

In [16]:
df['Cabin'].fillna(0, inplace=True)

for i in range(len(df)):
    if df.iloc[i]['Cabin'] != 0:
        df.at[i, 'Cabin'] = 1

Drop Ticket feature

In [18]:
df.drop('Ticket', axis=1, inplace=True)

Impute missing 'Age' values based on medians grouped by Pclass, sex_male, and whether 'Mister' is in passenger title

In [20]:
grouped_df = df.groupby(by=['Pclass', 'sex_male'], as_index=False).median()
grouped_df[['Pclass', 'sex_male', 'Age']]

,Pclass,sex_male,Age
0,1,0,36.0
1,1,1,42.0
2,2,0,28.0
3,2,1,29.5
4,3,0,22.0
5,3,1,25.0


In [22]:
def impute_age(df):
    
    df.Age.fillna(0, inplace=True)
    
    for i in range(len(df)):
        
        if 'Master' in df.iloc[i]['Name']:
            df.at[i,'Age'] = random.randint(1,10)
        
        elif 'Master' not in df.iloc[i]['Name']:
        
            if df.iloc[i]['Pclass'] == 1 and df.iloc[i]['Age'] == 0:
                if df.iloc[i]['sex_male'] == 0:
                    df.at[i, 'Age'] = grouped_df.iloc[0]['Age']
                else:
                    df.at[i, 'Age'] = grouped_df.iloc[1]['Age']

            elif df.iloc[i]['Pclass'] == 2 and df.iloc[i]['Age'] == 0:
                if df.iloc[i]['sex_male'] == 0:
                    df.at[i, 'Age'] = grouped_df.iloc[2]['Age']
                else:
                    df.at[i, 'Age'] = grouped_df.iloc[3]['Age']

            elif df.iloc[i]['Pclass'] == 3 and df.iloc[i]['Age'] == 0:
                if df.iloc[i]['sex_male'] == 0:
                    df.at[i, 'Age'] = grouped_df.iloc[4]['Age']
                else:
                    df.at[i, 'Age'] = grouped_df.iloc[5]['Age']

impute_age(df)

Extract titles from names

In [24]:
def title_from_name(df):
    df['title'] = None
    
    for i in range(len(df)):
        
        if 'Mr.' in df.iloc[i]['Name']:
            df.at[i, 'title'] = 'Mr'
        elif 'Mrs.' in df.iloc[i]['Name']:
            df.at[i, 'title'] = 'Mrs'
        elif 'Miss' in df.iloc[i]['Name']:
            df.at[i, 'title'] = 'Miss'
        elif 'Master' in df.iloc[i]['Name']:
            df.at[i, 'title'] = 'Master'
        elif 'Rev.' in df.iloc[i]['Name']:
            df.at[i, 'title'] = 'Rev'
        else:
            df.at[i, 'title'] = 'Other'
            
title_from_name(df)

,PassengerId,Pclass,Name,Age,SibSp,Parch,Fare,Cabin,sex_male,emb_C,emb_Q,emb_S,title
0,1,3,"Braund, Mr. Owen Harris",22.0,1,0,7.2500,0,1,0,0,1,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,71.2833,1,0,1,0,0,Mrs
2,3,3,"Heikkinen, Miss. Laina",26.0,0,0,7.9250,0,0,0,0,1,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,53.1000,1,0,0,0,1,Mrs
4,5,3,"Allen, Mr. William Henry",35.0,0,0,8.0500,0,1,0,0,1,Mr
